In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import traceback
import os

from ansys.mapdl.core import launch_mapdl

In [2]:
try:
    mapdl = launch_mapdl(start_instance=False, port=50052)
except:
    mapdl = launch_mapdl(start_instance=True, port=50052)

In [5]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   19:20:52  NOV 28, 2024 CP=      0.344

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [6]:
n_cell = 3
relative_density = 0.5
external_wall = False
three_dim = False

In [7]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000

t_over_l = np.sqrt(3)/2 - np.sqrt(3 - 4*relative_density)/2
cell_size = design_height/(np.sqrt(3)*n_cell + t_over_l)
wall_thickness = cell_size*t_over_l
external_wall_thickness = wall_thickness

side_length = cell_size - 1/np.sqrt(3)*wall_thickness
design_part = mapdl.rectng(-1/1000, design_width, 0, design_height)

for i in range(n_cell):
    j = 0
    while True:
        center = [cell_size/2 - np.sqrt(3)/2*wall_thickness + 2*j*cell_size - (i%2)*cell_size,
                (cell_size + 1/np.sqrt(3)*wall_thickness)*np.sqrt(3)/2 + i*np.sqrt(3)*cell_size]
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if center[0] - side_length < design_width:
            k_list = []
            for k in range(6):
                k_list.append(mapdl.k(x = center[0] + side_length*np.cos(np.pi/3*k), y = center[1] + side_length*np.sin(np.pi/3*k)))
            hole = mapdl.a(*k_list)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                print(f"error2 at i={i}, j={j}")
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if cell_size + 2*j*cell_size - (i%2)*cell_size < design_width:
            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size - 1/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error2 at i={i}, j={j}")
                print(e)

            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size + 3/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error3 at i={i}, j={j}")
                print(e)

        if cell_size/2 - 1/np.sqrt(3)*wall_thickness + 2*j*cell_size > design_width:
            break

        j += 1

mapdl.rectng(x1=-total_width/2+design_width/2, x2=0, y1=0, y2=design_height)
mapdl.rectng(design_width, total_width/2+design_width/2, 0, design_height)
mapdl.aadd("all")

if three_dim:
    mapdl.vext("ALL", dz = total_z)

In [21]:
mapdl.asel("all")

array([4])

In [27]:
mapdl.asum()
mapdl.get(entity="AREA", entnum=0, item1="AREA")

0.00289950014

In [41]:
from functions import kagome_horizontal
import numpy as np

external_wall = False
three_dim = False

total_density = pd.DataFrame([])
for n_cell in range(2, 10):
    density_dict = {}

    for relative_density in np.arange(0.05, 0.5, 0.05):
        design_width = 60/1000
        design_height = 25/1000
        total_width = 144/1000
        total_z = 5/1000

        mapdl.finish()
        mapdl.clear()
        mapdl.prep7()

        kagome_horizontal(mapdl, n_cell, relative_density, external_wall, three_dim)
        mapdl.asel("all")
        mapdl.asum()
        density = mapdl.get(entity="AREA", entnum=0, item1="AREA")/total_width/design_height
        density_dict[relative_density] = density
    total_density[n_cell] = density_dict

error2 at i=0, j=17
error2 at i=2, j=17
error2 at i=4, j=17
error2 at i=6, j=17


MapdlRuntimeError: 

Error in instance GRPC_127.0.0.1:50052

ADD AREAS
      INPUT AREAS =     1    2    3    4    5    6    7
      INPUT AREAS WILL BE DELETED IF POSSIBLE
 *** ERROR ***                           CP =    2207.562   TIME= 21:40:02
 Boolean operation failed.  Try adjusting the tolerance value on the     
 BTOL command to some fraction of the minimum keypoint distance.Model    
 Size (current problem) 1.515526e-01, current BTOL setting               
 1.000000e-05, minimum KPT distance 1.638762e-04.

In [ ]:
total_density.to_csv('temp.csv')

In [36]:
total_density = pd.DataFrame([])

In [34]:
total_density['new'] = [1, 2, 3]

In [37]:
density = pd.Series({1: 2, 'a': 1})

In [38]:
total_density['new'] = density